In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
x= np.loadtxt("data_x.txt",dtype=int)
y= np.loadtxt("data_y.txt",dtype=int)
data = np.loadtxt("data.txt",dtype=int)
n = sum(sum(data))
print(f"n = {n}")

n = 200


In [20]:
df = pd.DataFrame(data, columns=[y[0], y[1], y[2], y[3]])
df.set_index(x, inplace=True)

df

,15,25,30,35
10,15,0,0,0
20,10,80,30,0
30,0,0,45,20


In [21]:
df_new = df.div(n)
df_new['P(x=xi*)'] = df_new.sum(axis=1)
df_new.loc['qj*'] = df_new.sum(axis=0)
df_new

,15,25,30,35,P(x=xi*)
10,0.075,0.0,0.000,0.0,0.075
20,0.050,0.4,0.150,0.0,0.600
30,0.000,0.0,0.225,0.1,0.325
qj*,0.125,0.4,0.375,0.1,1.000


$\widetilde{M}(x) = \sum_{i=1}^n (x_i^*)p_i^*$

$\widetilde{D}(x) = \sum_{i=1}^n (x_i^*)^2p_i^* - (\widetilde{M}(x))^2$

$\widetilde{M}(y) = \sum_{j=1}^m (y_j^*)q_j^*$

$\widetilde{D}(y) = \sum_{j=1}^m (y_j^*)^2q_j^* - (\widetilde{M}(y))^2$

In [41]:
m_x = sum(index * row['P(x=xi*)'] if index != "qj*" else 0 for index, row in df_new.iterrows())
d_x = sum(index**2 * row['P(x=xi*)'] if index != "qj*" else 0 for index, row in df_new.iterrows()) - m_x**2
sigma_x = np.sqrt(d_x)
m_y = sum(column * df_new[column].iloc[-1] if column != "P(x=xi*)" else 0 for column in df_new.columns)
d_y = sum(column**2 * df_new[column].iloc[-1] if column != "P(x=xi*)" else 0 for column in df_new.columns) - m_y**2
sigma_y = np.sqrt(d_y)
   
print(f"m_x = {m_x}, d_x = {d_x}, sigma_x = {sigma_x:.3} m_y = {m_y}, d_y = {d_y}, sigma_y = {sigma_y:.3}")

m_x = 22.5, d_x = 33.75, sigma_x = 5.81 m_y = 26.625, d_y = 29.234375, sigma_y = 5.41


$\overline{k(x,y)} = \overline{cov(x, y)} = \sum_{i=1}^n\sum_{j=1}^m x_i^*y_j^*p_i^* - \widetilde{M}(x)\widetilde{M}(y)$

In [37]:
k = sum((index) * (column) * row[column] if index != "qj*" and column != "P(x=xi*)" else 0 for column in df_new.columns for index, row in df_new.iterrows()) - m_x * m_y
k

24.6875

$\widetilde{r_{xy}} = \cfrac{\widetilde{k(x,y)}}{\widetilde{\sigma}(x)\widetilde{\sigma}(y)}$

In [43]:
r = k / (sigma_x * sigma_y)
print(f"r = {r:.3}")

r = 0.786
